In [1]:
from underthesea import text_normalize

In [1]:
import py_vncorenlp

In [2]:
model = py_vncorenlp.VnCoreNLP(save_dir="/workspace/nlplab/quannd/scada-full-stack/chatbot-scada/vncorenlp")

2024-01-12 10:02:26 INFO  WordSegmenter:24 - Loading Word Segmentation model
2024-01-12 10:02:26 INFO  PosTagger:23 - Loading POS Tagging model
2024-01-12 10:02:30 INFO  NerRecognizer:34 - Loading NER model
2024-01-12 10:02:47 INFO  DependencyParser:32 - Loading Dependency Parsing model


In [4]:
text = "Tôi hôm nay dậy làm từ rất sớm."
text = text_normalize(text)
words = " ".join(model.word_segment(text)).split(" ")
segmented_text = " ".join(words)
words = [word.replace("_", " ") for word in words]
text = " ".join(words)
print(text)
print(words)
running_offset = 0
for word in words:
    try:
        word_offset = text.index(word, running_offset)
        word_len = len(word)
        running_offset = word_offset + word_len
        print(word,": ", word_offset, "->", running_offset)
    except:
        print(f"error at word: {word} and substring: {text[running_offset:]}")


Tôi hôm nay dậy làm từ rất sớm .
['Tôi', 'hôm nay', 'dậy', 'làm', 'từ', 'rất', 'sớm', '.']
Tôi :  0 -> 3
hôm nay :  4 -> 11
dậy :  12 -> 15
làm :  16 -> 19
từ :  20 -> 22
rất :  23 -> 26
sớm :  27 -> 30
. :  31 -> 32


In [5]:
segmented_text

'Tôi hôm_nay dậy làm từ rất sớm .'

In [17]:
from underthesea import dependency_parse, chunk, pos_tag

In [18]:
pos_tag("vẽ hình chữ nhật có chiều dài 20.4 cm. Hình này nằm ở góc trái trên màn hình.")

[('vẽ', 'V'),
 ('hình chữ nhật', 'N'),
 ('có', 'V'),
 ('chiều', 'N'),
 ('dài', 'A'),
 ('20.4 cm', 'N'),
 ('.', 'CH'),
 ('Hình', 'N'),
 ('này', 'P'),
 ('nằm', 'V'),
 ('ở', 'E'),
 ('góc', 'N'),
 ('trái', 'A'),
 ('trên', 'E'),
 ('màn hình', 'N'),
 ('.', 'CH')]

In [10]:
dir(underthesea)

['__all__',
 '__author__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__email__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 'chunk',
 'classify',
 'corpus',
 'dependency_parse',
 'feature_engineering',
 'infile',
 'models',
 'ner',
 'os',
 'pipeline',
 'pos_tag',
 'sent_tokenize',
 'sentiment',
 'sys',
 'text_normalize',
 'util',
 'version_file',
 'version_info',
 'word_tokenize']

In [1]:
import torch
from transformers import AutoTokenizer, AutoModel
phobert_tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")
phobert_model = AutoModel.from_pretrained("vinai/phobert-base-v2")


/workspace/nlplab/quannd/rasa_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
phobert_tokenizer.bos_token_id

0

In [9]:
# import torch
# from transformers import BertTokenizer
# tokenizer = BertTokenizer.from_pretrained("vinai/phobert-base-v2")

In [9]:
# from transformers import TFBertModel

# model = TFBertModel.from_pretrained("vinai/phobert-base-v2", from_pt=True)

In [65]:
device = "cuda" if torch.cuda.is_available() else "cpu"
phobert_model = phobert_model.to(device)

In [10]:
phobert_tokenizer.pad_token

'<pad>'

In [11]:
sentence = "<s> " + segmented_text + " </s>"

encoded_info  = phobert_tokenizer.encode_plus(sentence,
                                      padding='max_length',
                                      add_special_tokens=False,
                                      max_length=16,
                                      return_tensors='np',
                                      return_token_type_ids=False,
                                      return_attention_mask=True)
encoded_input = encoded_info['input_ids'].flatten()

In [12]:
encoded_info

{'input_ids': array([[   0,  218, 1113, 1764,   47,   39,   59,  471,    5,    2,    1,
           1,    1,    1,    1,    1]]), 'attention_mask': array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]])}

In [13]:
phobert_tokenizer.convert_ids_to_tokens(encoded_input)

['<s>',
 'Tôi',
 'hôm_nay',
 'dậy',
 'làm',
 'từ',
 'rất',
 'sớm',
 '.',
 '</s>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>']

In [17]:
encoded_input

[0, 746, 2]

In [62]:
phobert_tokenizer.decode(encoded_input[4])

'l à m'

In [74]:
for token in encoded_input:
    print(phobert_tokenizer.decode(token))

< s >
T ô i
h ô m _ n a y
d ậ y
l à m
t ừ
r ấ t
s ớ m
.
< / s >
< p a d >
< p a d >
< p a d >
< p a d >
< p a d >
< p a d >


In [75]:
input_ids = torch.tensor([phobert_tokenizer.encode(sentence)])

with torch.no_grad():
    input_ids = input_ids.to(device)
    features = phobert_model(input_ids)  


In [76]:
features[0].shape

torch.Size([1, 12, 768])

In [77]:
features[0].squeeze(0).cpu().numpy()[1:-1].shape

(10, 768)

In [78]:
abc_str = {'alias': None, 'config': 'vinai/phobert-base-v2'}
print(type(abc_str) == dict)

True


In [11]:
from typing import Any, Text, Dict
def tmp_func(cls, config: Dict[Text, Any]) -> None:
    print(config)
    # for k, v in config.items():
    #     print(k, v)

tmp_func(4, "Afsa")

Afsa


In [12]:
tmp_dict = {'alias': 'abccccc'}
{**tmp_dict, 'name': 'quan'}

{'alias': 'abccccc', 'name': 'quan'}

In [9]:
import numpy as np
tmp_np = np.array([[1, 2, 4, 5],
                   [5, 4, 2, 4],
                   [4, 2, 3],
                   [8, 9]])

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (4,) + inhomogeneous part.

In [2]:
from scipy.sparse import spmatrix
sp_1 = 

ValueError: This class is not intended to be instantiated directly.

In [1]:
import scipy
scipy.__version__

'1.8.1'

In [3]:
pip install bpemb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 1.7 MB/s eta 0:00:0000:0100:02m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 10.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
from bpemb import BPEmb

In [5]:
model = BPEmb(
    lang="vi",
    dim=100,
    vs=10000,
    vs_fallback=True
)

downloading https://nlp.h-its.org/bpemb/vi/vi.wiki.bpe.vs10000.model


100%|██████████| 387772/387772 [00:00<00:00, 435577.12B/s]


downloading https://nlp.h-its.org/bpemb/vi/vi.wiki.bpe.vs10000.d100.w2v.bin.tar.gz


100%|██████████| 3774870/3774870 [00:01<00:00, 2506606.89B/s]


In [10]:
from typing import Text
import numpy as np

def _create_word_vector(document: Text) -> np.ndarray:
    """Creates a word vector from a text. Utility method."""
    encoded_ids = model.encode_ids(document)
    if encoded_ids:
        return model.vectors[encoded_ids[0]]

    return np.zeros((100,), dtype=np.float32)

In [11]:
tmp_sent = "Trên bến dưới thuyền"
tokens = tmp_sent.split(" ")
word_vectors = np.array(
    [_create_word_vector(document=token) for token in tokens]
    )

In [13]:
word_vectors.shape

(4, 100)

In [79]:
sent_1 = "hôm nay tôi đi chơi bóng bàn"
sent_2 = "hôm nay tôi đến rạp chiếu phim và ăn pizza"

sent_1 = text_normalize(sent_1)
words = " ".join(model.word_segment(sent_1)).split(" ")
segmented_sent_1 = " ".join(words)
words = [word.replace("_", " ") for word in words]
sent_1 = " ".join(words)
print(sent_1)
print(words)

sent_2 = text_normalize(sent_2)
words = " ".join(model.word_segment(sent_2)).split(" ")
segmented_sent_2 = " ".join(words)
words = [word.replace("_", " ") for word in words]
sent_2 = " ".join(words)
print(sent_2)
print(words)


hôm nay tôi đi chơi bóng bàn
['hôm nay', 'tôi', 'đi', 'chơi', 'bóng bàn']
hôm nay tôi đến rạp chiếu phim và ăn pizza
['hôm nay', 'tôi', 'đến', 'rạp', 'chiếu phim', 'và', 'ăn', 'pizza']


In [80]:
print(segmented_sent_1,",", segmented_sent_2)

hôm_nay tôi đi chơi bóng_bàn , hôm_nay tôi đến rạp chiếu_phim và ăn pizza


In [81]:
ids_1 = phobert_tokenizer.encode(segmented_sent_1)
ids_2 = phobert_tokenizer.encode(segmented_sent_2)

In [82]:
# phobert_tokenizer.convert_ids_to_tokens(ids_1)

In [83]:
max_len = 15


attn_mask_1 = [1] * len(ids_1) + [0] * (max_len - len(ids_1))
ids_1 += [phobert_tokenizer.pad_token_id] *(max_len - len(ids_1))

attn_mask_2 = [1] * len(ids_2) + [0] * (max_len - len(ids_2))
ids_2 += [phobert_tokenizer.pad_token_id] *(max_len - len(ids_2))


In [84]:
ids_1

[0, 1113, 70, 57, 379, 11240, 2, 1, 1, 1, 1, 1, 1, 1, 1]

In [85]:
attn_mask_1

[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]

In [86]:
inputs = [ids_1, ids_2]
attn_masks = [attn_mask_1, attn_mask_2]

In [87]:
from torch.utils.data import TensorDataset, SequentialSampler, DataLoader

In [110]:
batch_size = 32
inputs = torch.tensor(inputs)
attn_masks = torch.tensor(attn_masks)
        
data = TensorDataset(inputs, attn_masks)
sampler = SequentialSampler(data)
dataloader = DataLoader(data, sampler=sampler, batch_size=batch_size)

/tmp/ipykernel_2408076/3712379048.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = torch.tensor(inputs)
/tmp/ipykernel_2408076/3712379048.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attn_masks = torch.tensor(attn_masks)


In [108]:
sampler

In [106]:
device

'cuda'

In [92]:
from tqdm import tqdm

In [109]:
with torch.no_grad():
    for batch in tqdm(dataloader):
        inputs, attn_masks = batch
        print(inputs)
        print(attn_masks)
        inputs = inputs.to(device)
        attn_masks = attn_masks.to(device)
        features = phobert_model(inputs, attn_masks)
        

100%|██████████| 1/1 [00:00<00:00,  7.24it/s]

tensor([[    0,  1113,    70,    57,   379, 11240,     2,     1,     1,     1,
             1,     1,     1,     1,     1],
        [    0,  1113,    70,    30,  3752,  8458,     6,   203, 17080,     2,
             1,     1,     1,     1,     1]], device='cuda:0')
tensor([[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]], device='cuda:0')


In [101]:
features.last_hidden_state[0].shape

torch.Size([15, 768])

In [99]:
features.pooler_output.shape

torch.Size([2, 768])

In [2]:
tmp_arr_1 = ["abc", "agb", "gvc", "dxa"]
tmp_arr_2 = [1, 2, 3, 4]

list(zip(tmp_arr_1, tmp_arr_2))

[('abc', 1), ('agb', 2), ('gvc', 3), ('dxa', 4)]

In [3]:
from rasa.nlu.constants import DENSE_FEATURIZABLE_ATTRIBUTES
DENSE_FEATURIZABLE_ATTRIBUTES

['text', 'response', 'action_text']

In [4]:
from rasa.shared.nlu.constants import TEXT, ACTION_TEXT
print(TEXT, ACTION_TEXT)

text action_text


In [1]:
from underthesea import text_normalize
text_normalize("tọa độ")


'tọa độ'

In [1]:
import underthesea

In [4]:
sent_1 = "hôm nay tôi đi chơi bóng bàn"
sent_2 = "hôm nay tôi đến rạp chiếu phim và ăn pizza"

In [5]:
from underthesea import word_tokenize

In [6]:
word_tokenize(sent_1)

['hôm nay', 'tôi', 'đi', 'chơi', 'bóng', 'bàn']

In [1]:
import fasttext

from huggingface_hub import hf_hub_download

model_path = hf_hub_download(repo_id="facebook/fasttext-vi-vectors", filename="model.bin")
model = fasttext.load_model(model_path)
model.words

/workspace/nlplab/quannd/rasa_env_3.4/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
model.bin:   2%|▏         | 115M/7.24G [00:21<22:05, 5.38MB/s]   


KeyboardInterrupt: 

In [8]:
import vietokenizer
import py_vncorenlp
from underthesea import text_normalize
segmenter = py_vncorenlp.VnCoreNLP(save_dir="/workspace/nlplab/quannd/scada-full-stack/chatbot-scada/vncorenlp/")


In [1]:
text = "Nhấn  Delete  và làm sạch mọi thứ như bạn làm sạch tâm hồn ."
normalized_text = text_normalize(text)
words = segmenter(normalized_text)
words = [word.replace("_", " ") for word in words.split(" ")]
words = [text_normalize(word) for word in words]

NameError: name 'text_normalize' is not defined

In [14]:
words

['Tăng chiều',
 'dài',
 'của',
 'hình vuông',
 'màu',
 'đỏ',
 'có',
 'kích thước',
 '250x250 pixel',
 ',',
 'độ',
 'dày',
 'đường',
 'viền',
 '1',
 'pixel',
 ',',
 'nằm',
 'ở',
 'tọa độ',
 '(',
 '10',
 ',',
 '10',
 ')',
 'của',
 'bản vẽ',
 'lên',
 '15',
 'cm',
 '.']

In [2]:
text = "Nhấn  Delete  và làm sạch mọi thứ như bạn làm sạch tâm hồn ."

In [3]:
running_offset = 0
for word in words:
    try:
        word_offset = text.index(word, running_offset)
        word_len = len(word)
        running_offset = word_offset + word_len
        print(word,": ", word_offset, "->", running_offset)
    except:
        print(f"error at word: {word} and substring: {text[running_offset:]}")


NameError: name 'words' is not defined

try to test tokenizer in rasa

In [ ]:
try:
            running_offset = 0
            tokens = []
            for word in words:
                word_offset = text.index(word, running_offset)
                word_len = len(word)
                running_offset = word_offset + word_len
                tokens.append(Token(word, word_offset))

            return tokens
        except:
            from underthesea import text_normalize
            print(text)
            running_offset = 0
            tokens = []
            new_text = text_normalize(text)
            new_words = new_text.split(" ")

            for word in new_words:
                word_offset = new_text.index(word, running_offset)
                word_len = len(word)
                running_offset = word_offset + word_len
                tokens.append(Token(word, word_offset))

            return tokens


In [10]:
import yaml

x = open("/workspace/nlplab/quannd/x.yml", "r")
y = open("/workspace/nlplab/quannd/y.yml", "r")

x_yaml = yaml.load(x, Loader=yaml.FullLoader)
y_yaml = yaml.load(y, Loader=yaml.FullLoader)

In [11]:
x_yaml

[{'intent': 'quan', 'examples': '- fasf\n- afa'}]

In [12]:
y_yaml

[{'intent': 'quan', 'examples': '- re\n- wdas'}]

In [13]:
x_yaml + y_yaml

[{'intent': 'quan', 'examples': '- fasf\n- afa'},
 {'intent': 'quan', 'examples': '- re\n- wdas'}]